## Import

In [150]:
import random
import os
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

## 해보고 싶은 것

### 1. train을 많이 줘서 학습을 많이 시키기
### 2. 전처리를 더 해보기 - 메터데이터 이용하기, 등등
### 3. gru 모델을 더 좋은 것을 이용해 보고 싶다.

In [151]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [152]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3090'

In [153]:
torch.cuda.is_available()

True

In [154]:
torch.__version__

'2.0.1+cu118'

In [155]:
# torch.cuda.empty_cache()

## Hyperparameter Setting

In [156]:
CFG = {
    'TRAIN_WINDOW_SIZE':120, # 90일치로 학습  초기는 90일이였음 
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':20,
    'LEARNING_RATE':1e-4,
    'BATCH_SIZE':4096,
    'SEED':41
}

In [157]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

### 데이터 불러오기

In [158]:
train_data = pd.read_csv('E:/LG/LG_data/train.csv').drop(columns=['ID', '제품'])

In [159]:
brand_keyword_cnt = pd.read_csv('E:/LG/LG_data/brand_keyword_cnt.csv')

In [160]:
train_data.head(10)

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-C002-0007,B002-C003-0038,B002-00001,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,1,3,2,0,0,2,0
2,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,B002-C001-0003,B002-C002-0008,B002-C003-0044,B002-00002,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,7,21,16,...,0,0,0,0,0,0,0,0,0,0
7,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,15,16,23,...,0,0,0,0,0,0,0,0,0,1
9,B002-C001-0001,B002-C002-0001,B002-C003-0003,B002-00003,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 데이터 전처리

In [164]:
# # 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
# numeric_cols = train_data.columns[4:]
# # 칵 column의 min 및 max 계산
# min_values = train_data[numeric_cols].min(axis=1)
# max_values = train_data[numeric_cols].max(axis=1)
# # 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
# ranges = max_values - min_values
# ranges[ranges == 0] = 1
# # min-max scaling 수행
# train_data[numeric_cols] = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)
# # max와 min 값을 dictionary 형태로 저장
# scale_min_dict = min_values.to_dict()
# scale_max_dict = max_values.to_dict()

# 새로 시도해볼 전처리 방식 - 분모를 0이 되는 것을 방지

In [165]:
import pandas as pd

# 더 작은 값을 추가하여 분모가 0인 경우 방지하는 상수 정의
EPSILON = 1e-8

numeric_cols = train_data.columns[4:]

# 칵 column의 min 및 max 계산
min_values = train_data[numeric_cols].min(axis=1)
max_values = train_data[numeric_cols].max(axis=1)

# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 EPSILON 더해주기
ranges = max_values - min_values + EPSILON

# min-max scaling 수행
scaled_data = (train_data[numeric_cols].subtract(min_values, axis=0)).div(ranges, axis=0)

# 스케일링된 데이터로 업데이트
train_data[numeric_cols] = scaled_data

# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values.to_dict()
scale_max_dict = max_values.to_dict()


In [175]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [176]:
train_data.head()

,대분류,중분류,소분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,1,6,37,0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,2,7,43,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.0,0.0,0.222222,0.0
2,2,7,43,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3,2,7,43,1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
4,0,0,2,2,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0


In [180]:
# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '중분류', '소분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train_data[col])
    train_data[col] = label_encoder.transform(train_data[col])

In [183]:
import numpy as np
from tqdm import tqdm

def make_train_data(data, train_size, predict_size):
    num_rows = len(data)
    window_size = train_size + predict_size
    
    input_data = []
    target_data = []
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, 4:])
        
        for j in range(len(sales_data) - window_size + 1):
            window = sales_data[j : j + window_size]
            temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
            input_data.append(temp_data)
            target_data.append(window[train_size:])
    
    input_data = np.array(input_data)
    target_data = np.array(target_data)
    
    return input_data, target_data

def make_predict_data(data, train_size):
    num_rows = len(data)
    input_data = []
    
    for i in tqdm(range(num_rows)):
        encode_info = np.array(data.iloc[i, :4])
        sales_data = np.array(data.iloc[i, -train_size:])
        
        window = sales_data[-train_size:]
        temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
        input_data.append(temp_data)
    
    input_data = np.array(input_data)
    
    return input_data


In [185]:
import numpy as np
from tqdm import tqdm

def make_grouped_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
    '''
    대분류, 중분류, 소분류, 브랜드별로 그룹화된 학습 데이터 생성
    data: 일별 판매량 데이터 프레임
    train_size: 학습에 활용할 기간
    predict_size: 추론할 기간
    '''
    grouped_data = data.groupby(['대분류', '중분류', '소분류', '브랜드'])
    input_data = []
    target_data = []

    for group_key, group_df in tqdm(grouped_data):
        group_input, group_target = make_train_data(group_df, train_size, predict_size)
        input_data.append(group_input)
        target_data.append(group_target)

    input_data = np.concatenate(input_data, axis=0)
    target_data = np.concatenate(target_data, axis=0)

    return input_data, target_data

def make_grouped_predict_data(data , train_size=CFG['TRAIN_WINDOW_SIZE']):
    '''
    대분류, 중분류, 소분류, 브랜드별로 그룹화된 추론 데이터 생성
    data: 일별 판매량 데이터 프레임
    train_size: 추론을 위해 필요한 일별 판매량 기간
    '''
    grouped_data = data.groupby(['대분류', '중분류', '소분류', '브랜드'])
    input_data = []

    for group_key, group_df in tqdm(grouped_data):
        group_input = make_predict_data(group_df, train_size)
        input_data.append(group_input)

    input_data = np.concatenate(input_data, axis=0)

    return input_data

# 대분류, 중분류, 소분류, 브랜드로 그룹화된 학습 데이터 생성
grouped_input_data, grouped_target_data = make_grouped_train_data(data)

# 대분류, 중분류, 소분류, 브랜드로 그룹화된 추론 데이터 생성
grouped_predict_input_data = make_grouped_predict_data(data)


  0%|          | 0/5109 [00:00<?, ?it/s]

100%|██████████| 5109/5109 [00:33<00:00, 150.29it/s]


In [186]:
# def make_train_data(data, train_size=CFG['TRAIN_WINDOW_SIZE'], predict_size=CFG['PREDICT_SIZE']):
#     '''
#     학습 기간 블럭, 예측 기간 블럭의 세트로 데이터를 생성
#     data : 일별 판매량
#     train_size : 학습에 활용할 기간
#     predict_size : 추론할 기간
#     '''
#     num_rows = len(data)
#     window_size = train_size + predict_size
    
#     input_data = np.empty((num_rows * (len(data.columns) - window_size + 1), train_size, len(data.iloc[0, :4]) + 1))
#     target_data = np.empty((num_rows * (len(data.columns) - window_size + 1), predict_size))
    
#     for i in tqdm(range(num_rows)):
#         encode_info = np.array(data.iloc[i, :4])
#         sales_data = np.array(data.iloc[i, 4:])
        
#         for j in range(len(sales_data) - window_size + 1):
#             window = sales_data[j : j + window_size]
#             temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
#             input_data[i * (len(data.columns) - window_size + 1) + j] = temp_data
#             target_data[i * (len(data.columns) - window_size + 1) + j] = window[train_size:]
    
#     return input_data, target_data

In [187]:
# def make_predict_data(data, train_size=CFG['TRAIN_WINDOW_SIZE']):
#     '''
#     평가 데이터(Test Dataset)를 추론하기 위한 Input 데이터를 생성
#     data : 일별 판매량
#     train_size : 추론을 위해 필요한 일별 판매량 기간 (= 학습에 활용할 기간)
#     '''
#     num_rows = len(data)
    
#     input_data = np.empty((num_rows, train_size, len(data.iloc[0, :4]) + 1))
    
#     for i in tqdm(range(num_rows)):
#         encode_info = np.array(data.iloc[i, :4])
#         sales_data = np.array(data.iloc[i, -train_size:])
        
#         window = sales_data[-train_size : ]
#         temp_data = np.column_stack((np.tile(encode_info, (train_size, 1)), window[:train_size]))
#         input_data[i] = temp_data
    
#     return input_data

In [188]:
# train_input, train_target = make_train_data(train_data)
# test_input = make_predict_data(train_data)

In [189]:
data_len = len(grouped_input_data)
val_size = int(data_len * 0.1)

train_input = grouped_input_data[:-val_size]
train_target = grouped_target_data[:-val_size]
val_input = grouped_input_data[-val_size:]
val_target = grouped_target_data[-val_size:]

In [190]:
# # Train / Validation Split
# data_len = len(train_input)
# val_input = train_input[-int(data_len*0.2):]
# val_target = train_target[-int(data_len*0.2):]
# train_input = train_input[:-int(data_len*0.1)]
# train_target = train_target[:-int(data_len*0.1)]

# 아마 위에랑 똑같을 것이다.

In [191]:
# # Train/Validation Split 비율 설정
# validation_ratio = 0.1

# # 데이터 길이 계산
# data_len = len(train_input)

# # Validation 데이터 및 타겟 분할
# val_size = int(data_len * validation_ratio)
# val_input, val_target = train_input[-val_size:], train_target[-val_size:]

# # Train 데이터 및 타겟 분할
# train_input, train_target = train_input[:-val_size], train_target[:-val_size]

# # Train 데이터와 Validation 데이터 확인
# print("Train Input Data Shape:", train_input.shape)
# print("Train Target Data Shape:", train_target.shape)
# print("Validation Input Data Shape:", val_input.shape)
# print("Validation Target Data Shape:", val_target.shape)


In [192]:
train_input.shape, train_target.shape, val_input.shape, val_target.shape, test_input.shape

((4705029, 110, 5),
 (4705029, 21),
 (522781, 110, 5),
 (522781, 21),
 (15890, 110, 5))

### Custom Dataset

In [193]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.Y = Y
        
    def __getitem__(self, index):
        if self.Y is not None:
            return torch.Tensor(self.X[index]), torch.Tensor(self.Y[index])
        return torch.Tensor(self.X[index])
    
    def __len__(self):
        return len(self.X)

In [194]:
train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

### 모델 선언

## GRU 이용

In [195]:
# import torch.nn as nn
# import torch

# class BaseModel(nn.Module):
#     def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE']):
#         super(BaseModel, self).__init__()
#         self.hidden_size = hidden_size
#         self.num_layers = num_layers
#         self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
#         self.fc = nn.Sequential(
#             nn.Linear(hidden_size, hidden_size // 2),
#             nn.ReLU(),
#             nn.Dropout(),
#             nn.Linear(hidden_size // 2, output_size)
#         )

#         self.actv = nn.ReLU()

#     def forward(self, x):
#         # x shape: (B, TRAIN_WINDOW_SIZE, 5)
#         batch_size = x.size(0)
#         hidden = self.init_hidden(batch_size, x.device)

#         # GRU layer
#         gru_out, hidden = self.gru(x, hidden)

#         # Only use the last output sequence
#         last_output = gru_out[:, -1, :]

#         # Fully connected layer
#         output = self.actv(self.fc(last_output))

#         return output.squeeze(1)

#     def init_hidden(self, batch_size, device):
#         # Initialize hidden state for all GRU layers
#         return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)


## 업그레이드 GRU

In [196]:
import torch.nn as nn
import torch

class ImprovedModel(nn.Module):
    def __init__(self, input_size=5, hidden_size=512, num_layers=2, output_size=CFG['PREDICT_SIZE']):
        super(ImprovedModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.gru = nn.GRU(input_size, hidden_size, num_layers=num_layers, batch_first=True)
        self.dropout = nn.Dropout(0.2)  # Adding dropout after GRU
        self.fc = nn.Sequential(
            nn.Linear(hidden_size, hidden_size // 2),
            nn.ReLU(),
            nn.Linear(hidden_size // 2, output_size)
        )
        self.actv = nn.LeakyReLU()  # Using LeakyReLU activation

    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size, x.device)

        gru_out, hidden = self.gru(x, hidden)
        gru_out = self.dropout(gru_out)  # Applying dropout

        last_output = gru_out[:, -1, :]

        output = self.actv(self.fc(last_output))

        return output.squeeze(1)

    def init_hidden(self, batch_size, device):
        return torch.zeros(self.num_layers, batch_size, self.hidden_size, device=device)

### 모델 학습

In [197]:
def train(model, optimizer, train_loader, val_loader, device):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None
    
    
    
    for epoch in range(1, CFG['EPOCHS']+1):
        model.train()
        train_loss = []
        train_mae = []
        
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            
            optimizer.zero_grad()
            
            output = model(X)
            loss = criterion(output, Y)
            
            loss.backward()
            optimizer.step()
            
            train_loss.append(loss.item())
        
        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')
        

        # 학습 루프 안에서
        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')

        # 학습이 끝난 후
    return best_model  # 모든 모델을 반환


In [198]:
def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []
    
    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)
            
            output = model(X)
            loss = criterion(output, Y)
            
            val_loss.append(loss.item())
    return np.mean(val_loss)

## Run !!

In [199]:
model = ImprovedModel() # BaseModel() 
optimizer = torch.optim.RAdam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/1149 [00:00<?, ?it/s]


TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

## 모델 추론

In [60]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [61]:
def inference(model, test_loader, device):
    predictions = []
    
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            
            output = model(X)
            
            # 모델 출력인 output을 CPU로 이동하고 numpy 배열로 변환
            output = output.cpu().numpy()
            
            predictions.extend(output)
    
    return np.array(predictions)

In [62]:
pred = inference(infer_model, test_loader, device)

100%|██████████| 4/4 [00:00<00:00,  7.38it/s]


In [63]:
# 추론 결과를 inverse scaling
for idx in range(len(pred)):
    pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]
    
# 결과 후처리
pred = np.round(pred, 0).astype(int)

In [64]:
pred.shape

(15890, 21)

## Submission

In [65]:
submit = pd.read_csv('E:/LG/LG_data/sample_submission.csv')
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [66]:
submit.iloc[:,1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,1,1,1,1
3,3,0,0,0,0,0,0,0,0,1,...,1,1,1,1,1,1,1,1,1,1
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,1,1


In [67]:
submit.to_csv('E:/LG/LG_data/models/baseline_submit_epoch_15_90_4096_512_RAdam_GRU_LAYER_2_V_2_T_1_e-4_e-888.csv', index=False)